In [5]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
embeddings


tensor([[-0.0720,  0.8865,  0.0067,  ..., -0.2312, -0.2527,  0.4367],
        [ 0.1011,  0.6739, -0.2718,  ..., -0.5791, -0.8743,  0.5991],
        [ 0.0120,  0.3784, -0.2010,  ..., -0.4346, -0.2772,  0.5504],
        [ 0.2595,  0.3794, -0.2860,  ..., -0.3827, -0.5138,  0.4720]],
       grad_fn=<DivBackward0>)

In [2]:
# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())


[[90.79100799560547, 71.17613220214844], [72.53498840332031, 90.14604187011719]]


In [4]:
embeddings

tensor([[-0.0045,  0.0549,  0.0004,  ..., -0.0143, -0.0156,  0.0270],
        [ 0.0058,  0.0388, -0.0156,  ..., -0.0333, -0.0503,  0.0345],
        [ 0.0010,  0.0307, -0.0163,  ..., -0.0352, -0.0225,  0.0446],
        [ 0.0225,  0.0329, -0.0248,  ..., -0.0332, -0.0445,  0.0409]],
       grad_fn=<DivBackward0>)